<p style="padding: 10px; border: 1px solid black;">
<img src="images/mlu-logo.png" alt="drawing" width="400"/> <br/>


# <a name="0">MLU Getting Started with Bedrock and Prompt Engineering</a>
## <a name="0">Amazon Bedrock API Walkthrough</a>

<br>

---

### Important Notes

1. Before running this notebook, make sure you have already granted access to the Bedrock model(s) used here, by following the instructions in the `Lab Setup` session at the course portal.

2. The Titan models undergo regular updates. Therefore, the returned results in this notebook may be different from the video recordings provided by MLU for this course.

---
    
Amazon Bedrock is a fully managed service that makes foundation models (FMs) from Amazon and third-party model providers easily accessible through an API. This notebook covers the Amazon Bedrock API using SDK for Python (Boto3).

Topics:

* __Accessing the Bedrock service using the SDK for Python (Boto3)__
* __Performing Bedrock API call: Basic API call and different options for customization__

### 1. Accessing the Bedrock service using the SDK for Python

We can access the Bedrock service through boto3 by providing the service name, region name and endpoint URL.

In [ ]:
import json, boto3

session = boto3.session.Session()

bedrock = session.client(
    service_name="bedrock",
    region_name=session.region_name,
)

Let's take a look at the available Large Language Models (LLMs) in Bedrock. As the list is long, here we print just the first three models, but you can print all by removing `[0:3]` form the code below

In [ ]:
bedrock.list_foundation_models()["modelSummaries"][0:3]

We use the service name 'bedrock-runtime' for inference. 

In [ ]:
# For inference
bedrock_inference = session.client(
    service_name="bedrock-runtime",
)

Model IDs can be used to select a specific model in the API calls. 

This demo is focusing on the __Amazon Titan Text G1 - Premier__ model from __Amazon__. We will use the model with id: `amazon.titan-text-premier-v1:0`

---

### 2. Bedrock API Call

Bedrock API has the following parameters:
* __`body`:__ The message body that includes the input text and model parameters. Model parameters help customize the models and the generated outputs.
* __`modelId`:__ Identifier of the model. We can pick a model from the list of models printed in the previous code block.
* __`accept`:__ The desired type of the inference body in the response.
* __`contentType`:__ The type of the input data in the request body.

<br>

__Model parameters:__

These parameters are provided within the body of the API call. Basically, we can control the randomness and the length of the generated sequences. 

__Randomness:__ 

* __`temperature`:__ Controls the randomness of the generated sequences. This parameter is between zero and one. When set closer to zero, the model tends to select higher probability words. When set further away from zero, the model may select lower-probability words. Temperature of zero gives the same output for the same input at every run.
* __`topP`:__ Top P defines a cut-off based on the sum of probabilities of the potential choices. With this cut-off, the model only selects from the most probable words whose probabilities sum up to the topP value. 

__Length:__ 

* __`maxTokens`:__ Controls the maximum number of tokens in the generated response.
* __`stopSequences`:__ A sequence of characters to stop the model from generating its output.

Let's define a function that will build our prompt as well as pass some model parameters to Bedrock. Once the call parameters are ready, we can use the __invoke_model()__ function to send the data and collect the response from the model. Then, we return the response at the end.

In [ ]:
def send_prompt(prompt_data, temperature=0.0, top_p=1.0, max_token_count=1000):

    body = json.dumps(
        {
            "inputText": prompt_data,
            "textGenerationConfig": {
                "temperature": temperature,
                "topP": top_p,
                "maxTokenCount": max_token_count,
            },
        }
    )

    modelId = "amazon.titan-text-premier-v1:0"
    accept = "application/json"
    contentType = "application/json"

    response = bedrock_inference.invoke_model(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )

    response_body = json.loads(response["body"].read())

    return response_body["results"][0]["outputText"]

Let's construct a simple API call and run it. 

As a sample text input, we use the following: __"Can you name a few real-life applications of natural language processing?"__. 

For inference parameters, we set the __`temperature`__ to 0.

In [ ]:
from IPython.display import Markdown, display

prompt_data = (
    """Can you name a few real-life applications of natural language processing?"""
)

display(Markdown(prompt_data))
display(Markdown(send_prompt(prompt_data, temperature=0.0)))

This provides a list of real-life NLP applications. 

If we want to generate slightly different looking outputs, we can increase the __`temperature`__ value. Let's also set the __`maxTokens`__ parameter this time.

Changing the temperature parameter will create a slightly different looking list.

In [ ]:
prompt_data = (
    """Can you name a few real-life applications of natural language processing?"""
)

display(Markdown(prompt_data))
display(Markdown(send_prompt(prompt_data, temperature=0.5, max_token_count=450)))

In addition to the __`temperature`__ and __`maxTokens`__ parameters, we can also add in the __`topP`__ parameter to set a cut-off for the sum of the probabilities of the potential words.

In [ ]:
prompt_data = (
    """Can you name a few real-life applications of natural language processing?"""
)

display(Markdown(prompt_data))
display(
    Markdown(send_prompt(prompt_data, max_token_count=450, temperature=0.5, top_p=0.7))
)

# Thank you!

<p style="padding: 10px; border: 1px solid black;">
<img src="images/mlu-logo.png" alt="drawing" width="400"/> <br/>